# Preprocessing 

In [1]:
import warnings
import time
import os
from sklearn.impute import SimpleImputer

import dask
from dask.distributed import Client
import geopandas as gpd
import numpy as np
import pandas as pd

/tmp/ipykernel_1771099/3511113298.py:8: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


This notebook will be used to prepare our feature and ground-truth data for our modeling process. At this point, we have a directory of .feather files that contain our features (see this notebook for feature generation). We want to concatentate these together, and perform the necessary operations to achieve a dataframe where a row is one observation unit; its columns are the associated features and the ground truth data that we want to train the model on. Broadly, we want our dataframe to take on the following form:

| observation_unit |  outcome_1 | outcome_2 | feature_1| feature_2 | feature_3
| ----|  ---- | ---- | -- | -- | -- |
| 1   |  $y_{1}$ | $y_{2}$ | $x_{1}$ | $x_{2}$ | $x_{3}$
| 2   |  $y_{1}$ | $y_{2}$ | $x_{1}$| $x_{2}$ | $x_{3}$

Where we can run regress `outcome_1` and `outcome_2` on our features, using the following form:

$y_{1}$ = $\beta_{1}$$x_{1}$ + $\beta_{2}$$x_{2}$ + $\beta_{3}$$x_{3}$ + $\beta_{n}$$x_{n}$ 

In our case, an observation is one SEA/year. We have ground truth data that summarizes the total crop yield, total crop loss, as well as other pertinent agricultural variables. We want our dataframe described above to look like this:

| SEA | Year | Observed Yield (Tonnes) | Observed Loss (Tonnes) | feature1| feature2 | feature3
| ----| ----| ---- | ---- | -- | -- | -- |
| 1   | 2016 | 72 | 13 | 1.23 | 3.25 | 0.123
| 2   | 2016  | 50 | 7.5 | 0.78| 1.2 | 2.4

In [5]:
# define the file paths for each file
feature_dir = "/capstone/mosaiks/repos/modeling/data/features_zmb_save/" #insert path to feature .feather files here

# get a list of all feather files in the directory
feather_files = [f for f in os.listdir(feature_dir) if f.endswith('.feather')]

# read in each file as a dataframe and concatenate them together
df = pd.concat([pd.read_feather(os.path.join(feature_dir, f)) for f in feather_files])

# print the resulting dataframe
print(df)

FileNotFoundError: [Errno 2] No such file or directory: '/capstone/mosaiks/repos/modeling/data/features_zmb_save/'

### Read in Features

In [3]:
# Let's read in the new concatenated features:
features = pd.read_feather("/capstone/mosaiks/repos/modeling/data/cropmosaiks_features_landsat8.feather")
features = features.drop(columns=['level_0', 'index'])

In [4]:
features

,0,1,2,3,4,5,6,7,8,9,...,994,995,996,997,998,999,lon,lat,year,month
0,3.245117,5.734004,0.0,0.0,0.025423,1.326510,6.456960,0.000000,0.008385,0.000000,...,0.000000,3.676284,1.050056,0.005223,2.868862,9.096745,22.144878,-16.384232,2013,4
1,1.944154,3.558259,0.0,0.0,0.006166,0.641115,3.980383,0.000000,0.001047,0.000000,...,0.026306,2.887371,0.955308,0.117883,2.254850,6.422407,22.124878,-16.384232,2013,4
2,3.311014,5.355516,0.0,0.0,0.009611,1.193801,5.894898,0.000000,0.006474,0.000000,...,0.000000,3.487587,0.957761,0.005029,2.811308,8.471640,22.134878,-16.384232,2013,4
3,3.193799,5.550329,0.0,0.0,0.034844,1.276846,6.246958,0.000000,0.009118,0.000000,...,0.000000,3.623213,1.066875,0.006653,2.810880,8.867989,22.134878,-16.394232,2013,4
4,1.610958,5.466636,0.0,0.0,0.092130,1.212386,6.973500,0.000000,0.000818,0.000000,...,0.000000,3.776426,1.608718,0.004816,2.745148,9.501865,22.104878,-16.324232,2013,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706824,1.888587,6.375369,0.0,0.0,0.000000,2.136316,6.739666,0.000000,0.000000,0.362333,...,0.000000,5.053378,0.081824,0.267609,2.998518,9.514809,28.724878,-16.404232,2017,12
706825,1.757204,7.121776,0.0,0.0,0.000000,2.536723,7.448850,0.000000,0.000000,0.577109,...,0.000000,5.623606,0.049325,0.345218,3.262203,10.222257,28.704878,-16.414232,2017,12
706826,1.868492,7.233674,0.0,0.0,0.000000,2.541375,7.580592,0.000000,0.000000,0.599434,...,0.000000,5.641362,0.040216,0.342799,3.250101,10.400089,28.714878,-16.414232,2017,12
706827,1.610369,6.393692,0.0,0.0,0.000000,2.195518,6.686218,0.000000,0.000000,0.531410,...,0.000000,5.199398,0.047613,0.430099,3.042479,9.429357,28.704878,-16.424232,2017,12


### Read in Ground-Truth Data


In [5]:
# Name list because gpd doesn't read in column names correctly
names = ["sea_unq", "year", "total_area_planted_ha", "total_area_harv_ha", "total_area_lost_ha",
         "total_harv_yield_kg", "avg_yield_kgha", "frac_area_harv", "frac_area_loss",
         "area_lost_animal_bird_destruction", "area_lost_floods_heavy_rain", "area_lost_na",
         "area_lost_wilting_due_to_drought", "area_lost_water_logging", "area_lost_soil_generally_bad",
         "area_lost_pests_and_diseases", "area_lost_fire", "area_lost_lack_of_fertilizer", "maize",
         "groundnuts", "frac_loss_drought", "geometry"]


country_sea = gpd.read_file('/capstone/mosaiks/repos/preprocessing/ground_data_clean/total2.shp', encoding = "utf-8")

country_sea.columns = names
country_sea.info

ERROR 1: PROJ: proj_create_from_database: Open of /Users/andrewbartnik/.conda/envs/mosaiks/share/proj failed


<bound method DataFrame.info of       sea_unq    year  total_area_planted_ha  total_area_harv_ha  \
0           1  2008.0                43.1300             42.2550   
1           1  2009.0                34.9725             34.9725   
2           1  2010.0                32.2150             26.7625   
3           1  2011.0                60.4075             59.7525   
4           1  2012.0                84.6175             74.6025   
...       ...     ...                    ...                 ...   
4381      392  2016.0              3272.0000           1416.0000   
4382      392  2017.0               402.0000             48.7500   
4383      392  2019.0               103.5250             66.5010   
4384      392  2020.0               103.5250             66.5010   
4385      392  2021.0                41.6625             32.4100   

      total_area_lost_ha  total_harv_yield_kg  avg_yield_kgha  frac_area_harv  \
0                 0.8750              28923.0      684.487043        0

In [6]:
# Filter country_sea for unique values of 'seq_unq' and 'geometry' - this will be useful for later
sea_unq_join = country_sea[['sea_unq', 'geometry']].drop_duplicates()

### Spatially join features to Ground-Truth Data


In [7]:
# Organize the features by growing season
# Carry months October, November, and December over to the following year's data
# These months represent the start of the growing season for the following year's maize yield
year_end = 2022

features['year'] = np.where(
    features['month'].isin([10, 11, 12]),
    features['year'] + 1, 
    features['year'])

features_gs = features[features['year'] <= year_end]
features_gs.sort_values(['year', 'month'], inplace=True)

We first create a GeoDataFrame (`features_new_gdf`) from the DataFrame features, using the longitude and latitude columns to create the geometry column (geospatial points) and specifying the CRS (Coordinate Reference System) as EPSG:4326.

In [8]:
# Create a geodataframe of the new features 
features_new_gdf = gpd.GeoDataFrame(
    features_gs,
    geometry=gpd.points_from_xy(x=features_gs.lon, y=features_gs.lat),
    crs='EPSG:4326'
)

Since our ground-truth data has an annual resolution, we need to pivot our monthly features wider so that each observation contains all of the features for the full year. The main goal in this next step is to aggregate the features data into years while preserving the monthly information as separate columns. To achieve this, the code performs the following steps:

1. Separate the 'geometry' column temporarily to avoid duplicating it during unstacking.
2. Create a multi-level index using 'lon', 'lat', 'year', and 'month' in the DataFrame.
3. Unstack the DataFrame, transforming the innermost index level ('month') into new columns, creating a wide format DataFrame where each row represents a unique combination of 'lon', 'lat', and 'year', and the columns contain the values of the original features for each month.
4. Merge the 'geometry' column back into the DataFrame.

The resulting DataFrame contains the features aggregated by year with separate columns for each month's data. The naming convention will be `featurenumber_monthnumber`, so `587_12` corresponds to the 587th feature generated in December for that year 

In [9]:
# Store the 'geometry' column separately before unstacking
geometry_col = features_new_gdf[['lon', 'lat', 'geometry']].drop_duplicates(subset=['lon', 'lat'])

# Perform the unstacking operation without the 'geometry' column
features_gs_no_geometry = features_gs.drop(columns=['geometry'])
features = features_gs_no_geometry.set_index(['lon', 'lat', 'year', 'month']).unstack()
features.columns = features.columns.map(lambda x: '{}_{}'.format(*x))

# Merge the 'geometry' column back into the features DataFrame
features = features.reset_index().merge(geometry_col, on=['lon', 'lat'])

In [10]:
features

,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,0_7,...,999_4,999_5,999_6,999_7,999_8,999_9,999_10,999_11,999_12,geometry
0,22.074878,-14.864232,2013,NaN,NaN,NaN,3.281571,3.122000,2.830094,1.736953,...,6.262064,6.879942,8.165108,8.109184,6.778838,6.895915,NaN,NaN,NaN,POINT (22.07488 -14.86423)
1,22.074878,-14.864232,2014,NaN,NaN,NaN,3.755434,3.512662,3.021393,1.995631,...,6.013897,6.966921,8.030674,7.918297,7.840030,7.432459,7.146350,7.551476,NaN,POINT (22.07488 -14.86423)
2,22.074878,-14.864232,2015,NaN,NaN,2.995817,NaN,3.692163,2.820786,1.555138,...,NaN,7.578374,7.890974,7.843058,7.666124,8.032577,3.168300,NaN,NaN,POINT (22.07488 -14.86423)
3,22.074878,-14.864232,2016,NaN,NaN,NaN,NaN,3.839774,3.176257,0.946849,...,NaN,6.993670,7.603345,7.367076,4.530905,7.415284,8.228379,7.735545,NaN,POINT (22.07488 -14.86423)
4,22.074878,-14.864232,2017,NaN,NaN,NaN,NaN,3.069077,2.404626,1.867290,...,NaN,6.132421,6.313315,6.858294,7.086604,7.594971,3.947087,NaN,NaN,POINT (22.07488 -14.86423)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114866,33.524878,-10.324232,2014,NaN,NaN,NaN,NaN,NaN,NaN,1.919606,...,NaN,NaN,NaN,8.378854,8.723665,NaN,8.814014,NaN,NaN,POINT (33.52488 -10.32423)
114867,33.524878,-10.324232,2015,NaN,3.548424,NaN,NaN,2.919775,2.627197,NaN,...,NaN,7.882391,8.462551,NaN,9.004691,9.066623,9.163157,9.252953,9.0457,POINT (33.52488 -10.32423)
114868,33.524878,-10.324232,2016,NaN,3.800390,4.274516,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.699384,8.858060,8.156837,NaN,POINT (33.52488 -10.32423)
114869,33.524878,-10.324232,2017,NaN,NaN,NaN,NaN,2.469667,NaN,1.501727,...,NaN,6.290899,NaN,6.962409,NaN,7.470372,8.842872,9.489412,NaN,POINT (33.52488 -10.32423)


In [11]:
# Convert the 'features' DataFrame to a GeoDataFrame
features_gdf = gpd.GeoDataFrame(features, geometry=features['geometry'], crs='EPSG:4326')

In [12]:
# Now lets combine the sea data 
spatial_join = gpd.sjoin(features_gdf, sea_unq_join, how='right', predicate = 'within')

In [13]:
features_join = spatial_join.merge(country_sea, on=['year', 'sea_unq'], how='inner')

In [14]:
# Drop the redundant independent lon and lat columns because now that they are in a separate geometry column
features_join = features_join.drop(['lon', 'lat', 'geometry_x'], axis = 1)


In [15]:
spatial_join

,index_left,lon,lat,year,0_1,0_2,0_3,0_4,0_5,0_6,...,999_5,999_6,999_7,999_8,999_9,999_10,999_11,999_12,sea_unq,geometry
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"POLYGON ((27.82327 -13.65772, 27.82294 -13.657..."
12,45659.0,27.944878,-13.514232,2014.0,NaN,NaN,NaN,NaN,3.497604,NaN,...,8.006833,NaN,NaN,NaN,inf,NaN,NaN,NaN,2,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
12,45856.0,27.954878,-13.514232,2013.0,NaN,NaN,NaN,4.459554,NaN,2.257166,...,NaN,7.202317,6.188143,2.896053,7.844928,NaN,NaN,NaN,2,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
12,45857.0,27.954878,-13.514232,2014.0,NaN,NaN,NaN,NaN,3.118880,2.216208,...,7.468665,7.485701,NaN,NaN,inf,NaN,NaN,NaN,2,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
12,45858.0,27.954878,-13.514232,2015.0,NaN,NaN,4.534863,NaN,0.000000,2.549744,...,inf,7.462369,inf,inf,7.907071,inf,6.892038,NaN,2,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4379,1371.0,23.234878,-16.274232,2014.0,NaN,NaN,3.130354,NaN,2.054446,1.594262,...,6.805032,7.486336,7.782812,8.521929,3.714375,4.583190,NaN,NaN,392,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
4379,1370.0,23.234878,-16.274232,2013.0,NaN,NaN,NaN,0.000000,2.773523,2.540741,...,7.896548,7.937504,8.383616,9.600647,9.939509,NaN,NaN,NaN,392,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
4379,1535.0,23.254878,-16.274232,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.302197,NaN,NaN,392,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
4379,1532.0,23.254878,-16.274232,2015.0,2.045358,2.10626,1.619694,1.427462,1.247452,0.891464,...,4.890819,5.193933,5.549318,5.899466,6.662761,5.685717,NaN,NaN,392,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


In [18]:
features_join

,index_left,year,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,...,area_lost_wilting_due_to_drought,area_lost_water_logging,area_lost_soil_generally_bad,area_lost_pests_and_diseases,area_lost_fire,area_lost_lack_of_fertilizer,maize,groundnuts,frac_loss_drought,geometry_y
0,45659.0,2014.0,NaN,NaN,NaN,NaN,3.497604,NaN,NaN,NaN,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
1,45857.0,2014.0,NaN,NaN,NaN,NaN,3.118880,2.216208,NaN,NaN,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
2,47247.0,2014.0,NaN,NaN,NaN,NaN,3.220327,2.319621,1.552471,1.111978,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
3,47029.0,2014.0,NaN,NaN,NaN,NaN,3.287071,2.063801,1.845703,1.002841,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
4,47527.0,2014.0,NaN,NaN,NaN,NaN,2.777567,2.310176,1.530833,1.445750,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,1364.0,2013.0,NaN,NaN,NaN,0.0,2.613850,2.360134,2.125934,1.361458,...,0.21720,0.0,2.0025,0.0,0.0,0.0,9.510600,0.287800,0.022062,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
2261,1524.0,2013.0,NaN,NaN,NaN,0.0,2.989810,2.522306,2.043034,1.269730,...,0.21720,0.0,2.0025,0.0,0.0,0.0,9.510600,0.287800,0.022062,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
2262,1424.0,2013.0,NaN,NaN,NaN,0.0,0.633685,2.360322,2.004860,1.566064,...,0.21720,0.0,2.0025,0.0,0.0,0.0,9.510600,0.287800,0.022062,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
2263,1530.0,2013.0,NaN,NaN,NaN,0.0,1.314045,1.064457,0.698342,0.594810,...,0.21720,0.0,2.0025,0.0,0.0,0.0,9.510600,0.287800,0.022062,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


In [19]:
features_join

,index_left,year,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,...,area_lost_wilting_due_to_drought,area_lost_water_logging,area_lost_soil_generally_bad,area_lost_pests_and_diseases,area_lost_fire,area_lost_lack_of_fertilizer,maize,groundnuts,frac_loss_drought,geometry_y
0,45659.0,2014.0,NaN,NaN,NaN,NaN,3.497604,NaN,NaN,NaN,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
1,45857.0,2014.0,NaN,NaN,NaN,NaN,3.118880,2.216208,NaN,NaN,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
2,47247.0,2014.0,NaN,NaN,NaN,NaN,3.220327,2.319621,1.552471,1.111978,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
3,47029.0,2014.0,NaN,NaN,NaN,NaN,3.287071,2.063801,1.845703,1.002841,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
4,47527.0,2014.0,NaN,NaN,NaN,NaN,2.777567,2.310176,1.530833,1.445750,...,24.28116,0.0,0.0000,0.0,0.0,0.0,134.207682,149.676522,0.081741,"POLYGON ((27.99349 -13.46497, 27.99352 -13.464..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,1364.0,2013.0,NaN,NaN,NaN,0.0,2.613850,2.360134,2.125934,1.361458,...,0.21720,0.0,2.0025,0.0,0.0,0.0,9.510600,0.287800,0.022062,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
2261,1524.0,2013.0,NaN,NaN,NaN,0.0,2.989810,2.522306,2.043034,1.269730,...,0.21720,0.0,2.0025,0.0,0.0,0.0,9.510600,0.287800,0.022062,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
2262,1424.0,2013.0,NaN,NaN,NaN,0.0,0.633685,2.360322,2.004860,1.566064,...,0.21720,0.0,2.0025,0.0,0.0,0.0,9.510600,0.287800,0.022062,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."
2263,1530.0,2013.0,NaN,NaN,NaN,0.0,1.314045,1.064457,0.698342,0.594810,...,0.21720,0.0,2.0025,0.0,0.0,0.0,9.510600,0.287800,0.022062,"POLYGON ((23.23962 -16.31204, 23.23876 -16.312..."


## Imputation

### Impute missing values

Imputing "manually" by descending group levels imputes NA values in multiple "cascading" steps, decreasing the proportion of inputed values with each step. First, the NA values are imputed at by both `year` and `geometry`, which should yield imputed values that most closely match the feature values that would be present in the data if there was no clouds obscuring the satellite images. Next, the remaining NA values that could not be imputed by both `year` and `district` are imputed by only `district`. Lastly, the remaining NA vlaues that could not be imputed by both `year` and `district` or by just `district` are imputed by `year` only. This option gives the user more control and transparency over how the imputation is executed.

Imputing using `scikit learn`'s simple imputer executes standard imputation, the details of which can be found in the `scikitlearn` documentation [here.](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

The imputation approach depends on the selection made at the top of this notebook for `impute_manual`.

In [20]:
impute_manual = False

# Extract the size of the features dataframe
rows, cols = features_join.shape

# compute the number of feature cells in the features dataframe
num_cells = rows * cols
num_cells

27232095

In [22]:
class bcolors:
    BL = '\x1b[1;34m' #GREEN
    GR = '\x1b[1;36m' #GREEN
    YL = '\x1b[1;33m' #YELLOW
    RD = '\x1b[1;31m' #RED
    RESET = '\033[0m' #RESET COLOR

In [23]:
# Notes: Have to change the year, get an error rn. Also, check to make sure the number of cells is correct
if impute_manual:
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Starting total row count: {bcolors.BL}{ln_ft}{bcolors.RESET}',
          f'\nPre-Impute NaN row count: {bcolors.RD}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPre-Impute NaN row %: {bcolors.RD}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPre-Impute NaN cell %: {bcolors.RD}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 1: Filling NaN values by month, year, and district group average')
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['year', 'sea_unq'], as_index=False) 
                .transform('mean')
               )
    )
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 1 NaN row count: {bcolors.YL}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 1 NaN row %: {bcolors.YL}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 1 NaN cell %: {bcolors.YL}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 2: Filling NaN values by month and district across group average')
    features_join = (
        features_join
        .fillna(features_join
                .groupby(['sea_unq'], as_index=False)
                .transform('mean')
               )
    )
    ln_ft = len(features_join)
    ln_na = len(features_join.dropna())
    print(f'Post step 2 NaN row count: {bcolors.GR}{ln_ft - ln_na}{bcolors.RESET}',
          f'\nPost step 2 NaN row %: {bcolors.GR}{((ln_ft - ln_na) / ln_ft)*100:.02f}{bcolors.RESET}',
          f'\nPost step 2 NaN cell %: {bcolors.GR}{(features_join.isna().sum().sum() / num_cells)*100:.02f}{bcolors.RESET}',
          f'\n\nStep 3: Drop remaining NaN values\n')
    features_join = features_join.dropna(axis=0)
    print(f'Ending total row count: {bcolors.BL}{len(features_join)}{bcolors.RESET}')
else:
    # Store the geometry column separately
    geometry_col = features_join['geometry_y']
    # Remove the geometry column from the DataFrame
    features_join = features_join.drop(columns=['geometry_y'])
    features_join = features_join.set_index(['year', 'sea_unq'])
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer.fit_transform(features_join)
    features_join[:] = imputer.transform(features_join)
    features_join = features_join.reset_index()
    # Add the geometry column back to the DataFrame
    features_join['geometry_y'] = geometry_col

ValueError: Input X contains infinity or a value too large for dtype('float64').